In [ ]:
pip install langchain openai fastapi uvicorn sqlite3

In [ ]:
#!pip install langchain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "APIKEY"

In [11]:
import os, getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [12]:
#Create a Tool (Example: get_weather)

from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Returns a mock weather report for the given city."""
    return f"It's always sunny in {city} with 25°C!"

In [13]:
#Initialize GPT Model (via LangChain)

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1", model_provider="openai")

In [14]:
#Create the ReAct Agent (with Tool Binding)
from langgraph.prebuilt import create_react_agent

tools = [get_weather]
agent_executor = create_react_agent(model=model, tools=tools)

In [15]:
# Run the Agent Without Memory (Single Turn)

input_message = {"role": "user", "content": "What is the weather in Pune?"}
response = agent_executor.invoke({"messages": [input_message]})

for msg in response["messages"]:
    msg.pretty_print()



================================ Human Message =================================

What is the weather in Pune?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_O6LVkzn1psF9tMVqnQcIwFhc)
 Call ID: call_O6LVkzn1psF9tMVqnQcIwFhc
  Args:
    city: Pune
================================= Tool Message =================================
Name: get_weather

It's always sunny in Pune with 25°C!
================================== Ai Message ==================================

The weather in Pune is sunny with a temperature of 25°C.


In [21]:
#Add Memory for Multi-Turn Agent

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model=model, tools=tools, checkpointer=memory)

config = {"configurable": {"thread_id": "weather-001"}}

# Turn 1
agent_executor.invoke({"messages": [{"role": "user", "content": "My city is Nagpur"}]}, config)

# Turn 2
agent_executor.invoke({"messages": [{"role": "user", "content": "What's the weather where I live?"}]}, config)

{'messages': [HumanMessage(content='My city is Nagpur', additional_kwargs={}, response_metadata={}, id='7f9af95d-d578-4afd-9604-7918be1ae277'),
  AIMessage(content='Great! How can I assist you regarding your city, Nagpur? Would you like to know something specific—like the weather, tourist attractions, news, or anything else? Let me know how I can help!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 52, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BlxLDEmcz8LcsR3LRfmVjbJqM9lSh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a47ad429-2426-4102-b211-5c8b42c6dd0b-0', usage_metadata={'input_tokens': 52, 'output_tokens': 44

In [17]:
# Turn 2
agent_executor.invoke({"messages": [{"role": "user", "content": "What's the weather where I live?"}]}, config)

{'messages': [HumanMessage(content='My city is Nagpur', additional_kwargs={}, response_metadata={}, id='79353d6b-9235-4611-8359-69315213a816'),
  AIMessage(content='Thanks for sharing! How can I assist you related to Nagpur? For example, would you like to know about the weather, places to visit, current news, or something else?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 52, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BlxHop29k08qfwkUYPJVN2UIIbOsY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--30bbe6a9-85cf-4fff-ad2c-d3925d4d8ec1-0', usage_metadata={'input_tokens': 52, 'output_tokens': 38, 'total_tokens': 90, 'in

In [22]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1", model_provider="openai")

query = {
    "role": "user",
    "content": "Walmart's sales increased by 20% this month. Think step-by-step about possible causes."
}

response = model.invoke([query])
print(response.text())

Sure! Let’s break down possible causes for **Walmart’s 20% sales increase this month**—step by step:

---

### 1. **Internal Factors**

#### **a. Promotions & Discounts**
- **Major sales events**: Did Walmart run large promotions (e.g., Memorial Day, early back-to-school) or flash sales?
- **Discounts or price matches**: Lower prices can drive volume.

#### **b. New Product Launches**
- Did Walmart introduce highly anticipated products or exclusive merchandise?
- Extended private-label offerings (like Great Value) might boost sales.

#### **c. Expansion**
- Opened new stores or revamped existing ones.
- Enhanced online delivery to more locations.

#### **d. E-commerce Growth**
- Improved app/website usability, new delivery/pickup options, or digital marketing.
- Expanded grocery pickup/delivery may have drawn more online orders.

#### **e. Loyalty Programs**
- Reward or membership programs incentivized spending.

---

### 2. **External Factors**

#### **a. Macroeconomic Shifts**
- **St

What is Plan-and-Execute (PnE)?

Plan-and-Execute is a powerful agent pattern that:
•	First plans a sequence of subtasks based on the user request.
•	Then executes each step, often using tools, APIs, or functions.
This improves reliability, transparency, and modularity in AI task solving.

Use Case:
Prompt: “Launch a weekend sale campaign across 3 top-selling categories.”
 
Step 1: Plan
The agent first thinks through all the required steps before doing anything:

PLAN:
To launch a weekend sale campaign across the top 3 categories, the following steps are required:

1. Identify the top 3 selling product categories for the last 30 days.
2. Retrieve current pricing and inventory levels for these categories.
3. Draft a weekend sale campaign with discounts and promo copy.
4. Submit the campaign for manager approval.

Step 2: Execute Each Step
Each planned step is then executed one by one, using appropriate tools or API functions:

✅ Step 1: Identify top categories
🛠️ Tool Call → get_top_categories(last_n_days=30, top_k=3)

Response:
["Groceries", "Electronics", "Personal Care"]

✅ Step 2: Fetch pricing and stock
🛠️ Tool Call → get_pricing_and_inventory(["Groceries", "Electronics", "Personal Care"])
Response:
{
  "Groceries": {"avg_price": "$3.5", "in_stock": "High"},
  "Electronics": {"avg_price": "$120", "in_stock": "Moderate"},
  "Personal Care": {"avg_price": "$8", "in_stock": "High"}
}


✅ Step 3: Create campaign draft
🛠️ Tool Call → create_campaign_draft(...)
Response:
Drafted a weekend promo:  
🛒 Groceries – 15% off  
🔌 Electronics – 10% cashback  
🧼 Personal Care – Buy 1 Get 1 Free  

✅ Step 4: Submit for approval
🛠️ Tool Call → submit_campaign(draft_id="promo_2025_weekend_001")
Response:
✅ Campaign submitted for approval. Awaiting response from Marketing Lead.

✅ Final Response to User:
🎯 Campaign Planning & Execution Complete!

🔹 Categories Selected: Groceries, Electronics, Personal Care  
🔹 Campaign Draft: Discounts + Promo Copy Created  
🔹 Status: Submitted for approval (ID: promo_2025_weekend_001)

In [25]:
%%writefile weekend_sale_agent.py
#In Code (LangChain-style)
from langchain.agents import initialize_agent
from langchain.tools import tool
from langchain_community.chat_models import ChatOpenAI

# Define each tool step
@tool
def get_top_categories(last_n_days: int, top_k: int):
    """Return top-k selling categories in the last N days."""
    return ["Groceries", "Electronics", "Personal Care"]

@tool
def get_pricing_and_inventory(categories: list):
    """Return mock pricing and inventory details for given categories."""
    return {
        "Groceries": {"avg_price": "$3.5", "in_stock": "High"},
        "Electronics": {"avg_price": "$120", "in_stock": "Moderate"},
        "Personal Care": {"avg_price": "$8", "in_stock": "High"},
    }

@tool
def create_campaign_draft(data: dict):
    """Create a weekend sales campaign draft for the categories."""
    return "Drafted weekend sale promo with discounts."

@tool
def submit_campaign(draft_id: str):
    """Submit the campaign draft for approval."""
    return "Campaign submitted for approval."

# Combine tools
tools = [get_top_categories, get_pricing_and_inventory, create_campaign_draft, submit_campaign]

# Initialize agent
llm = ChatOpenAI(model="gpt-4", temperature=0.3)
agent = initialize_agent(tools, llm, agent_type="chat-zero-shot-react-description")

# Run
agent.run("Launch a weekend sale campaign across 3 top-selling categories.")

Overwriting weekend_sale_agent.py


In [26]:
%%writefile weekend_sale_agent1.py

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# ✅ Define your tools with docstrings
@tool
def get_top_categories() -> list:
    """Returns top 3 selling product categories."""
    return ["Groceries", "Electronics", "Personal Care"]

@tool
def get_pricing_and_inventory() -> dict:
    """Returns pricing and inventory levels."""
    return {
        "Groceries": {"avg_price": "$3.5", "in_stock": "High"},
        "Electronics": {"avg_price": "$120", "in_stock": "Moderate"},
        "Personal Care": {"avg_price": "$8", "in_stock": "High"},
    }

@tool
def create_campaign_draft() -> str:
    """Creates a campaign draft with promotions."""
    return """
    🛒 Groceries – 15% off
    🔌 Electronics – 10% cashback
    🧼 Personal Care – Buy 1 Get 1 Free
    """

@tool
def submit_campaign() -> str:
    """Submits the campaign for approval."""
    return "✅ Campaign submitted for approval. ID: promo_2025_weekend_001"

# ✅ Load GPT-4 model (or GPT-3.5 for free tier)
llm = ChatOpenAI(model="gpt-4", temperature=0.3)

# ✅ Register tools
tools = [get_top_categories, get_pricing_and_inventory, create_campaign_draft, submit_campaign]

# ✅ Create agent using LangGraph
memory = MemorySaver()
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# ✅ Set thread ID
config = {"configurable": {"thread_id": "campaign-001"}}

# ✅ Ask agent to plan and execute
input_msg = {"role": "user", "content": "Launch a weekend sale campaign across 3 top-selling categories."}

for step in agent_executor.stream({"messages": [input_msg]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

Writing weekend_sale_agent1.py


In [ ]:
%%writefile langchain_agent.py

from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from typing import TypedDict, List, Union
import os

OPEN_API_KEY = " APIKEY"  # Replace with your OpenAI API key

# 🔐 Set your OpenAI key
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY
# 1. Define tool
@tool
def calculator(expression: str) -> str:
    """Evaluates a basic arithmetic expression."""
    return str(eval(expression))

# 2. Define state schema
class AgentState(TypedDict):
    messages: List[HumanMessage]

# 3. Define LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 4. Wrap LLM with a lambda that extracts `messages` from the state
llm_node = RunnableLambda(lambda state: llm.invoke(state["messages"]))

# 5. Wrap tool with dummy logic (you can later route to actual tool parsing)
# tool_node = RunnableLambda(lambda state: {"messages": [HumanMessage(content=calculator("(45 * 23) + 129"))]})
# Tool node that evaluates the user's expression dynamically
tool_node = RunnableLambda(
    lambda state: {
        "messages": [
            HumanMessage(content=calculator(state["messages"][-1].content))
        ]
    }
)

# 6. Build graph
graph_builder = StateGraph(AgentState)
graph_builder.add_node("llm", llm_node)
graph_builder.add_node("calc_tool", tool_node)

graph_builder.set_entry_point("llm")
graph_builder.add_edge("llm", "calc_tool")
graph_builder.add_edge("calc_tool", END)

# 7. Compile
graph = graph_builder.compile()

# 8. Invoke the agent with a message
result = graph.invoke({"messages": [ HumanMessage(content="(99 + 1) * 2")]})
#print("🤖 Agent Final Response:", result)

Overwriting langchain_agent.py


In [44]:
%%writefile langchain_agent1.py

from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from typing import TypedDict
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents.agent_types import AgentType
from langchain.agents.initialize import initialize_agent
from langchain.agents.agent_toolkits.python.base import PythonREPLTool
from langchain_openai import ChatOpenAI

# Step 1: Define state schema (input is a string)
class AgentState(TypedDict):
    input: str
    output: str

# Step 2: LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Step 3: Tool (Python calculator)
calculator = PythonREPLTool()

# Step 4: Tool execution node (simple Runnable)
def run_calculator(state: AgentState) -> AgentState:
    expression = state["input"]
    result = calculator.invoke(expression)
    return {"input": expression, "output": result}

tool_node = RunnableLambda(run_calculator)

# Step 5: Build LangGraph
builder = StateGraph(AgentState)
builder.add_node("calculator", tool_node)
builder.set_entry_point("calculator")
builder.set_finish_point("calculator")
graph = builder.compile()

# Step 6: Invoke the agent
if __name__ == "__main__":
    query = "(99 + 1) * 2"
    result = graph.invoke({"input": query})
    print(f"🤖 Final Answer for '{query}': {result['output']}")

Writing langchain_agent1.py


In [ ]:
%%writefile walmart_mcp.py

# Walmart MCP Agent using LangChain and FastAPI
# This agent converts natural language queries into SQL queries using OpenAI's GPT-4o model
# and maintains session memory for context.

from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import JSONResponse
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import os, traceback
from pprint import pprint

OPEN_API_KEY = "API KEY"  # Replace with your OpenAI API key
# 🔐 Set API key
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY  # Use GPT-4o

app = FastAPI()

# LLM Model
llm = ChatOpenAI(temperature=0, model="gpt-4o")

# Request body
class QueryRequest(BaseModel):
    session_id: str
    query: str

# Session memory
session_store = {}

def get_memory(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

# Natural language → SQL template
def query_to_sql(messages):
    messages = messages["input"]
    user_input = messages[-1].content
    llm_messages = [
        HumanMessage(content=f"Convert the following natural language query to SQL: {user_input}")
    ]
    pprint(llm_messages)
    return llm.invoke(llm_messages) # Invoke the LLM to get SQL query

#Wrap with LangChain memory handler
chain = RunnableWithMessageHistory(
    RunnableLambda(query_to_sql),
    lambda session_id: get_memory(session_id),
    input_messages_key="input",
    history_messages_key="history",
)

# 🚀 Endpoint to handle Walmart agent queries
@app.post("/mcp")
async def handle_query(request: QueryRequest):
    try:
        session_id = request.session_id
        query = request.query

        print(f"[Walmart-MCP] Session: {session_id} | Query: {query}")

        response = await chain.ainvoke(
            {"input": [HumanMessage(content=query)]},  #
            config={"configurable": {"session_id": session_id}},
        )
        return {"response": response.content}

    except Exception as e:
        traceback.print_exc()
        return JSONResponse(status_code=500, content={"error": str(e)})

Overwriting walmart_mcp.py


In [ ]:
%%writefile walmart_mcp_schema.py

# Walmart MCP Agent with Schema-aware SQL Generation
# This agent uses LangChain to convert natural language queries into SQL queries
# while being aware of the database schema, enhancing accuracy and relevance.
# It maintains session memory for context and uses FastAPI for the web interface.
# Requirements: langchain, openai, fastapi, uvicorn, sqlite3
# LangChain SQL Generator Agent (Schema-aware)

from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import JSONResponse
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import os, traceback


OPEN_API_KEY = "API KEY"
# 1. Set API Key
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY  # Use GPT-4o

app = FastAPI()

# 2. Language Model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 3. Schema definition (for prompt context)
schema_context = """
Table: products
Columns: product_id, product_name, category, price, location, sales_volume

Table: customers
Columns: customer_id, name, city, age, gender
"""

# 4. Prompt Template (Prompt Engineering for SQL)
sql_prompt_template = PromptTemplate.from_template(
    """
You are an expert data analyst. Convert the user's natural language query into a SQL query.
Use the following table schema:
{schema}

Query: {user_query}
SQL:
"""
)

# 5. Request Body
class QueryRequest(BaseModel):
    session_id: str
    query: str

# 6. In-memory Session Store
session_store = {}

def get_memory(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

# 7. Tool Execution Chain
def generate_sql(messages):
    user_message = messages["input"][-1].content
    final_prompt = sql_prompt_template.format(schema=schema_context, user_query=user_message)
    return llm.invoke([HumanMessage(content=final_prompt)])

chain = RunnableWithMessageHistory(
    RunnableLambda(generate_sql),
    lambda session_id: get_memory(session_id),
    input_messages_key="input",
    history_messages_key="history",
)

# 8. Endpoint
@app.post("/generate-sql")
async def sql_agent(request: QueryRequest):
    try:
        session_id = request.session_id
        query = request.query

        response = await chain.ainvoke(
            {"input": [HumanMessage(content=query)]},
            config={"configurable": {"session_id": session_id}},
        )

        return {"sql_query": response.content}

    except Exception as e:
        traceback.print_exc()
        return JSONResponse(status_code=500, content={"error": str(e)})


from fastapi import Query

@app.get("/mcp")
async def handle_query_get(session_id: str = Query(...), query: str = Query(...)):
    try:
        print(f"[Walmart-MCP-GET] Session: {session_id} | Query: {query}")

        response = await chain.ainvoke(
            {"input": [HumanMessage(content=query)]},
            config={"configurable": {"session_id": session_id}},
        )
        return {"response": response.content}

    except Exception as e:
        traceback.print_exc()
        return JSONResponse(status_code=500, content={"error": str(e)})


Overwriting walmart_mcp_schema.py


http://127.0.0.1:8000/mcp?session_id=user001&query=Show%20top%205%20selling%20items%20in%20Pune

In [45]:
import langchain
import langchain_core
print("LangChain version:", langchain.__version__)
print("LangChain Core version:", langchain_core.__version__)

LangChain version: 0.3.26
LangChain Core version: 0.3.66


In [ ]:
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.python_repl.tool import PythonREPLTool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition

# 1. Define the state
def run_python_code(state):
    tool = PythonREPLTool()
    input_text = state["messages"][-1].content
    result = tool.invoke(input_text)
    return {"messages": [HumanMessage(content=str(result))]}

# 2. Convert tool to OpenAI-compatible function
py_tool = convert_to_openai_tool(run_python_code)

# 3. Create LLM and tool node
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_node = ToolNode(llm)

# 4. Create the graph
builder = StateGraph(schema={"messages": list})

builder.add_node("llm", llm_node)
builder.add_node("python", RunnableLambda(run_python_code))

# Conditional routing
builder.set_conditional_entry_point("llm", tools_condition)
builder.add_edge("llm", "python")
builder.add_edge("python", END)

graph = builder.compile()

# 5. Run the agent
result = graph.invoke({
    "messages": [HumanMessage(content="What is (45 * 23) + 129 ?")]
})

# 6. Print final result
final_message = result["messages"][-1].content
print("🤖 Agent Final Response:", final_message)

In [8]:
%%writefile langgraph_python_repl.py

from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.python_repl import PythonREPLTool

# Step 1: Define the shared state
class AgentState(dict):
    pass

# Step 2: Define the REPL tool
python_repl = PythonREPLTool()

# Step 3: Define a LangGraph node that uses the REPL
def run_python(state: AgentState) -> AgentState:
    question = state["messages"][-1].content
    print(f"💡 Executing: {question}")
    result = python_repl.invoke(question)
    return {"messages": state["messages"] + [AIMessage(content=result)]}

tool_node = RunnableLambda(run_python)

# Step 4: Build the LangGraph
builder = StateGraph(AgentState)
builder.add_node("python", tool_node)
builder.set_entry_point("python")
builder.set_finish_point("python")
graph = builder.compile()

# Step 5: Run it
result = graph.invoke({"messages": [HumanMessage(content="(45 * 23) + 129")]})
print("🤖 Agent Final Response:", result["messages"][-1].content)

Overwriting langgraph_python_repl.py


In [9]:
%%writefile langgraph_agent2.py

from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_community.tools.python_repl import PythonREPLTool

# Step 1: Define shared state
class AgentState(dict):
    pass

# Step 2: Initialize tools
llm = ChatOpenAI(temperature=0, model="gpt-4o")  # Or use Gemini if preferred
python_tool = PythonREPLTool()

# Step 3: Define logic nodes

# 3a. Reasoning node using LLM
def decide_action(state: AgentState) -> AgentState:
    user_msg = state["messages"][-1].content
    tool_prompt = f"""You are a smart assistant. Decide:

- If the question needs Python calculation, say: "use_tool"
- Else answer directly

Query: {user_msg}
Answer:"""
    decision = llm.invoke([HumanMessage(content=tool_prompt)])
    print(f"🤖 LLM Decision: {decision.content}")
    if "use_tool" in decision.content.lower():
        return {"messages": state["messages"], "use_tool": True}
    return {"messages": state["messages"] + [AIMessage(content=decision.content)], "use_tool": False}

# 3b. Tool execution node
def run_python(state: AgentState) -> AgentState:
    user_input = state["messages"][-1].content
    output = python_tool.invoke(user_input)
    return {"messages": state["messages"] + [AIMessage(content=output)]}

# Step 4: LangGraph setup
builder = StateGraph(AgentState)
builder.add_node("decide", RunnableLambda(decide_action))
builder.add_node("python", RunnableLambda(run_python))

# Conditional routing
def should_use_tool(state: AgentState) -> str:
    return "python" if state.get("use_tool") else END

builder.set_entry_point("decide")
builder.add_conditional_edges("decide", should_use_tool, {"python": "python", END: END})
builder.set_finish_point("python")

graph = builder.compile()

# Step 5: Run agent
result = graph.invoke({"messages": [HumanMessage(content="What is (45 * 23) + 129?")]})
print("🧠 Final Response:", result["messages"][-1].content)

Writing langgraph_agent2.py


In [10]:
import getpass
import os

# Set your OpenAI API key securely if not already set in environment
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("🔐 Enter your OpenAI API key: ")

# Import LangChain's chat model initializer
from langchain.chat_models import init_chat_model

# Initialize GPT-4.1 model from OpenAI
model = init_chat_model(
    model="gpt-4.1",
    model_provider="openai",     # Explicitly tell LangChain to use OpenAI
    temperature=0.0              # Optional: make outputs deterministic
)

# Test the model
response = model.invoke([{"role": "user", "content": "Hi there! Who are you?"}])
print(response.text())

Hello! I’m ChatGPT, an AI language model created by OpenAI. I’m here to help answer your questions, have conversations, and assist with a wide range of topics. How can I help you today?
